### Imports

In [44]:
import requests 
import json 
import pandas as pd
from bs4 import BeautifulSoup
from mplsoccer import Pitch, VerticalPitch
import numpy as np
from matplotlib.offsetbox import OffsetImage
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.image as mpimg
from matplotlib.patches import ConnectionPatch
import os

import urllib.request
from PIL import Image

import ScraperFC as sfs
from io import BytesIO

# Importing twitter keys
import tweepy
import sys
sys.path.append('/Users/emilio/Documents/1- Projects/dataAnalyst/football/data/twitter_bot')
from keys import api_key, api_secret, access_token, access_token_secret, bearer_token 

In [45]:
# Fonts and colors 
from matplotlib import font_manager
locations = ['/Users/emilio/Documents/3- Resources/lightstats/fonts/Montserrat']
font_files = font_manager.findSystemFonts(fontpaths=locations)
for file in font_files:
    font_manager.fontManager.addfont(file)
font = 'Montserrat'

# Define the colors using hexadecimal representations
background_color = '#F7F7F7' 
title_color = '#000000'   
text_color = '#333333'  
sub_text_color = '#808080'   
accent_color1 = '#4682B4'    
accent_color2 = '#ADD8E6' 

### Scrap info

In [46]:
# Scrap sofascore
sofascore = sfs.Sofascore()


sofascore_url = 'https://www.sofascore.com/real-madrid-fc-bayern-munchen/xdbsEgb#id:12173506'
sofascore_url = sofascore_url.replace('id:', '')


# Scrap fotmob
fotmob_url = 'https://www.fotmob.com/api/matchDetails?matchId=4446289'

payload = {}
headers = {}

response = requests.request("GET", fotmob_url, headers=headers, data=payload)

data = response.json()

# print(response.text) Imprime el html

### DataFrames

In [47]:
dfShots = pd.DataFrame(response.json()['content']['shotmap']['shots'])
df_matchStats = sofascore.get_general_match_stats(sofascore_url)

### Team's Stats

In [48]:
# Extracting the 'stats' dictionary from the 'expected_goals_stats' dictionary
expected_goals_stats = data['content']['stats']['Periods']['All']['stats'][2]['stats'][1]
# Extracting the array of expected goals values from the 'expected_goals_stats' dictionary
expected_goals_array = expected_goals_stats['stats']


# Lista de todas las estadísticas
all_stats = ['xG', 'Possesion', 'Shots', 'Fouls', 'Big Chances']

# Obtener los datos para el equipo local
local_data = {
    'xG': float(expected_goals_array[0]),
    'Possesion': float(df_matchStats[df_matchStats['name'] == 'Ball possession']['homeValue'].iloc[0]),
    'Shots': int(df_matchStats[df_matchStats['name'] == 'Total shots']['home'].iloc[0]),
    'Fouls': int(df_matchStats[df_matchStats['name'] == 'Fouls']['home'].iloc[0]),
    'Big Chances': float(df_matchStats[df_matchStats['name'] == 'Big chances']['home'].iloc[0])
}

# Obtener los datos para el equipo visitante
away_data = {
    'xG': float(expected_goals_array[1]),
    'Possesion': float(df_matchStats[df_matchStats['name'] == 'Ball possession']['awayValue'].iloc[0]),
    'Shots': int(df_matchStats[df_matchStats['name'] == 'Total shots']['away'].iloc[0]),
    'Fouls': int(df_matchStats[df_matchStats['name'] == 'Fouls']['away'].iloc[0]),
    'Big Chances': float(df_matchStats[df_matchStats['name'] == 'Big chances']['away'].iloc[0])
}

### Game Data

In [49]:
round = data['general']['leagueRoundName']
match_time = data['general']['matchTimeUTC']
season = data['general']['parentLeagueSeason']
score = data['header']['status']['scoreStr']
pen_flag = data['header']['status']['reason']['short']
league = sofascore.get_match_data(sofascore_url)['tournament']['uniqueTournament']['name']

if pen_flag == 'Pen':
    away_penalties = data['header']['status']['reason']['penalties'][1]
    local_penalties = data['header']['status']['reason']['penalties'][0]

### Team's Data

In [50]:
# Check if 'general' key exists in the response
local_name = data['header']['teams'][0]['name']
local_id = data['header']['teams'][0]['id']
local_color = dfShots[dfShots['teamId'] == local_id]['teamColor'].iloc[0]
local_img_url = data['header']['teams'][0]['imageUrl']

away_name = data['header']['teams'][1]['name']
away_id = data['header']['teams'][1]['id']
away_color = dfShots[dfShots['teamId'] == away_id]['teamColor'].iloc[0]
away_img_url = data['header']['teams'][1]['imageUrl']


team_data = {}

# Function to populate team data
def populate_team_data(team_id, data, df_shots):
    team_index = 0 if team_id == data['header']['teams'][0]['id'] else 1
    team_name = data['header']['teams'][team_index]['name']
    team_color = df_shots[df_shots['teamId'] == team_id]['teamColor'].iloc[0]
    team_img_url = data['header']['teams'][team_index]['imageUrl']
    return team_name, team_color, team_img_url

# Populate team data
local_name, local_color, local_img_url = populate_team_data(local_id, data, dfShots)
away_name, away_color, away_img_url = populate_team_data(away_id, data, dfShots)

### Shotmap

In [51]:
# Filtrar los disparos del equipo local y del equipo visitante
local_shots = dfShots[dfShots['teamId'] == local_id]
away_shots = dfShots[dfShots['teamId'] == away_id]

# Filtrar los goles del equipo local y mostrarlos de manera diferente
local_goal_shots = local_shots[local_shots['eventType'] == 'Goal']
away_goal_shots = away_shots[away_shots['eventType'] == 'Goal']

# Filtrar los disparos que no son goles del equipo local y mostrarlos con transparencia
local_non_goal_shots = local_shots[local_shots['eventType'] != 'Goal']
away_non_goal_shots = away_shots[away_shots['eventType'] != 'Goal']

### Player shots

#### Data

In [53]:
""" Print the names of the player that shoot """
away_players_shooting = away_shots['playerName'].unique()
local_players_shooting = local_shots['playerName'].unique()

print(local_players_shooting)
print(away_players_shooting)

['Leroy Sané' 'Harry Kane' 'Jamal Musiala' 'Leon Goretzka' 'Eric Dier'
 'Serge Gnabry']
['Vinicius Junior' 'Rodrygo' 'Nacho Fernandez' 'Toni Kroos'
 'Federico Valverde' 'Brahim Diaz']


In [54]:
#player_name = 'Marcus Rashford'
dfPom = data['content']['matchFacts']['playerOfTheMatch']
pom_name = dfPom['name']['fullName']

# If you want the shots of the POM use 'pom_name'
player_shots = dfShots[dfShots['playerName'] == pom_name]

player_goal_shots = player_shots[player_shots['eventType'] == 'Goal']
player_non_goal_shots = player_shots[player_shots['eventType'] != 'Goal']

In [55]:
playerId = str(player_shots['playerId'].unique()[0])
player_img_url = f'https://images.fotmob.com/image_resources/playerimages/{playerId}.png'

# Retrieve team data
player_team_id = player_shots['teamId'].unique()[0]

if player_team_id == local_id:
    against_id = away_id
    against_name, against_color, against_img_url = away_name, away_color, away_img_url
elif player_team_id == away_id:
    against_id = local_id
    against_name, against_color, against_img_url = local_name, local_color, local_img_url

#### Plot shots

In [56]:
""" Shots of the Selected Player during a Match """

def playerShots(season): 
    pitch = VerticalPitch(pitch_type='custom', pitch_length=105, pitch_width=68, half=True, pitch_color=background_color)
    fig, ax = pitch.draw()

    fig.patch.set_facecolor(background_color)

    if not player_goal_shots.empty:
        pitch.scatter(player_goal_shots.x, player_goal_shots.y, ax=ax, marker='football', s=200)
    if not player_non_goal_shots.empty:
        pitch.scatter(player_non_goal_shots.x, player_non_goal_shots.y, ax=ax, alpha=0.6, color='black')

    # Fetch and plot the local team image (logo)
    response = requests.get(player_img_url)
    img_data = Image.open(BytesIO(response.content))
    ax_logo = fig.add_axes([0, 0.95, 0.1, 0.1])  # [x, y, Width, Height]
    ax_logo.imshow(img_data)
    ax_logo.axis(False)

    fig.text(0.1, 1, f'{pom_name} shots vs {against_name}', fontsize=14, weight='bold', fontfamily=font, color=title_color)
    fig.text(0.1, 0.95, f'{local_name} vs {away_name} | {round}', fontsize=12, fontfamily=font, color=sub_text_color)

    fig.text(.25, 0.1, "@3zavalam", fontsize=10, ha='right', fontfamily=font, color=text_color)
    fig.text(.75, 0.1, " Data: SofaScore y FotMob", fontsize=10, ha='center', fontfamily=font, color=text_color)

    # Replace "/" with "-" in the season string
    season = season.replace("/", "-")
    # Create the directory path
    directory = os.path.join("output", "shots", "players", league, season, round)
    # Check if the directory exists, if not, create it
    if not os.path.exists(directory):
        os.makedirs(directory)
    # Define the file path
    file_path_player = os.path.join(directory, f"{pom_name}_shots_vs_{against_name}.png")
    plt.savefig(file_path_player, bbox_inches='tight') 
    plt.close()

    return file_path_player

### Team Shots

In [57]:
# Contar el número de own goals del equipo local y visitante
away_own_goals_count = away_shots[away_shots['isOwnGoal'] == True]['isOwnGoal'].count()
local_own_goals_count = local_shots[local_shots['isOwnGoal'] == True]['isOwnGoal'].count()

# Restar el número de own goals del equipo contrario
away_goals = (away_shots[away_shots['eventType'] == 'Goal']['eventType'].count() - away_own_goals_count) + local_own_goals_count
local_goals = (local_shots[local_shots['eventType'] == 'Goal']['eventType'].count() - local_own_goals_count) + away_own_goals_count

In [58]:
""" Shots of the Local team"""

def localShots(season):
    pitch = VerticalPitch(pitch_type='custom', pitch_length=105, pitch_width=68, half=True, pitch_color=background_color)
    fig, ax = pitch.draw()

    fig.patch.set_facecolor(background_color)

    if not local_goal_shots.empty:
        pitch.scatter(local_goal_shots.x, local_goal_shots.y, ax=ax, marker='football', s=200)
    if not local_non_goal_shots.empty:
        pitch.scatter(local_non_goal_shots.x, local_non_goal_shots.y, ax=ax, alpha=0.6, label=local_name, color=local_color)

    # Fetch and plot the local team image (logo)
    response = requests.get(local_img_url)
    img_data = Image.open(BytesIO(response.content))
    ax_logo = fig.add_axes([0, 0.95, 0.1, 0.1])  # [x, y, Width, Height]
    ax_logo.imshow(img_data)
    ax_logo.axis(False)

    fig.text(0.1, 1, f'{local_name} shots vs {away_name}', fontsize=16, weight='bold', fontfamily=font, color=text_color)
    fig.text(0.1, 0.95, f'{local_name} vs {away_name} | {round}', fontsize=13, fontfamily=font, color=sub_text_color)

    fig.text(.25, 0.1, "@3zavalam", fontsize=10, ha='right', fontfamily=font, color=text_color)
    fig.text(.75, 0.1, " Data: SofaScore y FotMob", fontsize=10, ha='center', fontfamily=font, color=text_color)

    # Data about the shots
    fig.text(.92, .6, f'{local_data['xG']}', fontsize=14, fontfamily=font, weight='bold', color=text_color)
    fig.text(1, .6, 'xG', fontfamily=font, color=text_color, fontsize=14)

    fig.text(.92, .5, f'{local_goals}', fontsize=14, fontfamily=font, weight='bold', color=text_color)
    fig.text(1, .5, 'Goals', fontfamily=font, color=text_color, fontsize=14)

    fig.text(.92, .4, f'{local_shots['eventType'].count()}', fontsize=14, fontfamily=font,  weight='bold', color=text_color)
    fig.text(1, .4, 'Shots', fontfamily=font, color=text_color, fontsize=14)
    
    # Replace "/" with "-" in the season string
    season = season.replace("/", "-")
    # Create the directory path
    directory = os.path.join("output", "shots", "teams", league, season, round)
    # Check if the directory exists, if not, create it
    if not os.path.exists(directory):
        os.makedirs(directory)
    # Define the file path
    file_path_localShots = os.path.join(directory, f"{local_name}_shots_vs_{away_name}.png")
    plt.savefig(file_path_localShots, bbox_inches='tight') 
    plt.close()

    return file_path_localShots

In [59]:
""" Shots of the Away team"""

def awayShots(season):
    pitch = VerticalPitch(pitch_type='custom', pitch_length=105, pitch_width=68, half=True, pitch_color=background_color)
    fig, ax = pitch.draw()

    fig.patch.set_facecolor(background_color)

    if not away_goal_shots.empty:
        pitch.scatter(away_goal_shots.x, away_goal_shots.y, ax=ax, marker='football', s=200)
    if not away_non_goal_shots.empty:
        pitch.scatter(away_non_goal_shots.x, away_non_goal_shots.y, ax=ax, alpha=0.6, label=away_name, color=away_color)

    # Fetch and plot the local team image (logo)
    response = requests.get(away_img_url)
    img_data = Image.open(BytesIO(response.content))
    ax_logo = fig.add_axes([0, 0.95, 0.1, 0.1])  # [x, y, Width, Height]
    ax_logo.imshow(img_data)
    ax_logo.axis(False)

    fig.text(0.1, 1, f'{away_name} shots vs {local_name}', fontsize=16, weight='bold', fontfamily=font, color=text_color)
    fig.text(0.1, 0.95, f'{local_name} vs {away_name} | {round}', fontsize=13, fontfamily=font, color=sub_text_color)

    fig.text(.25, 0.1, "@3zavalam", fontsize=10, ha='right', fontfamily=font, color=text_color)
    fig.text(.75, 0.1, " Data: SofaScore y FotMob", fontsize=10, ha='center', fontfamily=font, color=text_color)

    # Data about the shots
    fig.text(.92, .6, f"{away_data['xG']}", fontsize=14, fontfamily=font, weight='bold', color=text_color)
    fig.text(1, .6, 'xG', fontfamily=font, color=text_color, fontsize=14)

    fig.text(.92, .5, f'{away_goals}', fontsize=14, fontfamily=font, weight='bold', color=text_color)
    fig.text(1, .5, 'Goals', fontfamily=font, color=text_color, fontsize=14)

    fig.text(.92, .4, f'{away_shots['eventType'].count()}', fontsize=14, fontfamily=font,  weight='bold', color=text_color)
    fig.text(1, .4, 'Shots', fontfamily=font, color=text_color, fontsize=14)


    # Replace "/" with "-" in the season string
    season = season.replace("/", "-")
    # Create the directory path
    directory = os.path.join("output", "shots", "teams", league, season, round)
    # Check if the directory exists, if not, create it
    if not os.path.exists(directory):
        os.makedirs(directory)
    # Define the file path
    file_path_awayShots = os.path.join(directory, f"{away_name}_shots_vs_{local_name}.png")
    plt.savefig(file_path_awayShots, bbox_inches='tight') 
    plt.close()
    
    return file_path_awayShots

### Calling Functions

In [60]:
""" Uncomment the ones to plot """
file_path_player = playerShots(season=season)
file_path_localShots = localShots(season=season)
file_path_awayShots = awayShots(season=season)

file_paths = [file_path_player, file_path_localShots, file_path_awayShots]

### Twitter

In [61]:
def tweet(file_path, text):
        auth = tweepy.OAuthHandler(api_key, api_secret)
        auth.set_access_token(access_token, access_token_secret)
        api = tweepy.API(auth, wait_on_rate_limit=True)

        # V2 Twitter API Authentication
        client = tweepy.Client(
            bearer_token,
            api_key,
            api_secret,
            access_token,
            access_token_secret,
            wait_on_rate_limit=True,
        )

        media_id = api.media_upload(filename=file_path).media_id_string
        client.create_tweet(text=text, media_ids=[media_id])
        print("Tweeted!")

In [62]:
if 'Liga' in league or 'CONMEBOL' in league or 'CONCACAF' in league:
    tweet_text_awayShots = (f"Tiros {away_name} vs {local_name}")
    tweet_text_localShots = (f"Tiros {local_name} vs {away_name}")
    tweet_text_playerShots = (f"Tiros de {pom_name} vs {against_name}")
else:
    tweet_text_awayShots = (f"Shots {away_name} vs {local_name}")
    tweet_text_localShots = (f"Shots {local_name} vs {away_name}")
    tweet_text_playerShots = (f"Shots by {pom_name} vs {against_name}")

In [63]:
tweet(file_paths=file_paths, text=f'Shotmaps {local_name} vs {away_name}')

TypeError: tweet() got an unexpected keyword argument 'file_paths'